In [33]:
!pip install datasets
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
import torch
import pickle
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import re
from rouge_score import rouge_scorer
import random
import nltk 
import matplotlib.pyplot as plt

In [35]:
with open('/kaggle/input/dataset/qa_dataset_full.pkl', 'rb') as f:
    data = pickle.load(f)
    
    passages_train = data['train']['passages']
    questions_train = data['train']['questions']
    answers_train = data['train']['answers']
    
    passages_val = data['val']['passages']
    questions_val = data['val']['questions']
    answers_val = data['val']['answers']
    
    passages_test = data['test']['passages']
    questions_test = data['test']['questions']
    answers_test = data['test']['answers']

In [36]:
def prepare_dataset(passages, questions, answers):
    dataset_dict = []
    
    # Преобразуем данные в формат для трансформеров
    for i in range(len(questions)):
        try:
            question_id = questions['question_id'].iloc[i]
            passage_id = questions['passage_id'].iloc[i]
            question_text = questions['question_text'].iloc[i]
            
            # Находим соответствующий текст отрывка
            passage_row = passages[passages['passage_id'] == passage_id]
            if len(passage_row) == 0:
                continue
                
            passage_text = passage_row['passage_text'].iloc[0]
            
            # Находим ответ на этот вопрос
            answer_row = answers[(answers['passage_id'] == passage_id) & 
                                (answers['question_id'] == question_id)]
            
            if len(answer_row) == 0:
                continue
            
            answer_id = answer_row['answer_id'].iloc[0]

            # Удаляем номера и скобки в начале текста, если они есть
            clean_passage = re.sub(r'^\(\d+\)\s*', '', passage_text)
            
            # 1. Используем первые 50-70 символов как ответ
            short_answer = clean_passage[:min(70, len(clean_passage))]
            
            # 2. Ищем первое предложение
            sentence_match = re.search(r'[^.!?]+[.!?]', clean_passage)
            first_sentence = sentence_match.group(0) if sentence_match else short_answer
            
            # Выбираем ответ из стратегий
            answer_text = first_sentence.strip()
            
            start_idx = passage_text.find(answer_text)
            if start_idx == -1:
                start_idx = 0
            
            dataset_dict.append({
                'id': f"{passage_id}-{question_id}",
                'context': passage_text,
                'question': question_text,
                'answers': {
                    'text': [answer_text],
                    'answer_start': [start_idx]
                }
            })
            
        except Exception as e:
            print(f"Ошибка при обработке примера {i}: {e}")
            continue
    
    return Dataset.from_list(dataset_dict)

In [37]:
train_dataset = prepare_dataset(passages_train, questions_train, answers_train)
val_dataset = prepare_dataset(passages_val, questions_val, answers_val)
test_dataset = prepare_dataset(passages_test, questions_test, answers_test)

print(f"Размер обучающего набора: {len(train_dataset)}")
print(f"Размер валидационного набора: {len(val_dataset)}")
print(f"Размер тестового набора: {len(test_dataset)}")

Размер обучающего набора: 2897
Размер валидационного набора: 529
Размер тестового набора: 1813


In [38]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Маппинг между фичами и оригинальными примерами
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        sample_idx = sample_mapping[i]
        
        # Получаем ответ
        answer = examples["answers"][sample_idx]
        answer_text = answer["text"][0]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer_text)
        
        # Находим токены начала и конца ответа
        token_start_index = 0
        token_end_index = 0
        
        # Находим первый токен, содержащий начало ответа
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char and end > start_char:
                token_start_index = idx
                break
                
        # Находим последний токен, содержащий конец ответа
        for idx, (start, end) in enumerate(offsets):
            if start < end_char and end >= end_char:
                token_end_index = idx
                break
        
        # Если ответ не найден в токенах, устанавливаем CLS как ответ
        if token_start_index == 0 and token_end_index == 0:
            tokenized_examples["start_positions"].append(0)
            tokenized_examples["end_positions"].append(0)
        else:
            tokenized_examples["start_positions"].append(token_start_index)
            tokenized_examples["end_positions"].append(token_end_index)

    return tokenized_examples

In [40]:
#Токенизируем набор данных

tokenized_train = train_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_val = val_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=val_dataset.column_names
)


Map:   0%|          | 0/2897 [00:00<?, ? examples/s]

Map:   0%|          | 0/529 [00:00<?, ? examples/s]

In [41]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred # labels здесь могут не использоваться
    start_logits, end_logits = predictions

    predicted_answers = []
    reference_answers = []

    num_eval_samples = len(tokenized_val)
    eval_indices = range(num_eval_samples)
    original_indices = [tokenized_val[i]['overflow_to_sample_mapping'] for i in eval_indices] \
                       if 'overflow_to_sample_mapping' in tokenized_val.features else eval_indices
    unique_original_indices = sorted(list(set(original_indices)))
    examples_map = {i: val_dataset[i] for i in unique_original_indices}


    for i in eval_indices:
        start_logits_i = start_logits[i]
        end_logits_i = end_logits[i]

        start_idx = np.argmax(start_logits_i)
        end_idx = np.argmax(end_logits_i)

        if end_idx < start_idx:
             end_idx = start_idx 
        try:
            input_ids = tokenized_val[i]["input_ids"]
        except IndexError:
            print(f"Предупреждение: Индекс {i} вне диапазона для tokenized_val. Пропуск.")
            continue

        # Конвертируем предсказанные индексы токенов в текстовый ответ
        tokens = tokenizer.convert_ids_to_tokens(input_ids)
        valid_start_idx = min(start_idx, len(tokens) - 1)
        valid_end_idx = min(end_idx, len(tokens) - 1)

        if valid_end_idx < valid_start_idx:
             valid_end_idx = valid_start_idx

        answer_tokens = tokens[valid_start_idx : valid_end_idx + 1]
        answer = tokenizer.convert_tokens_to_string(answer_tokens)

        # Очищаем от специальных токенов
        answer = answer.replace(tokenizer.cls_token, "").replace(tokenizer.sep_token, "").strip()

        try:
            if 'overflow_to_sample_mapping' in tokenized_val.features:
                 original_sample_idx = tokenized_val[i]['overflow_to_sample_mapping']
            else:
                 original_sample_idx = i

            reference = examples_map[original_sample_idx]["answers"]["text"][0]
        except Exception as e:
            print(f"Предупреждение: Не удалось получить эталонный ответ для примера {i}. Ошибка: {e}")
            reference = ""

        predicted_answers.append(answer)
        reference_answers.append(reference)

    def calculate_f1(pred, ref):
        pred_tokens = set(pred.lower().split())
        ref_tokens = set(ref.lower().split())

        if not pred_tokens or not ref_tokens:
            return 0.0

        common_tokens = pred_tokens.intersection(ref_tokens)

        precision = len(common_tokens) / len(pred_tokens)
        recall = len(common_tokens) / len(ref_tokens)

        if precision + recall == 0:
            return 0.0

        f1 = 2 * precision * recall / (precision + recall)
        return f1

    f1_scores = [calculate_f1(pred, ref) for pred, ref in zip(predicted_answers, reference_answers)]
    f1_score = np.mean(f1_scores) if f1_scores else 0.0 # Используем numpy.mean для единообразия

    rouge_scores_collected = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    decoded_preds_rouge = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in predicted_answers]
    decoded_labels_rouge = ["\n".join(nltk.sent_tokenize(ref.strip())) for ref in reference_answers]

    for pred, ref in zip(decoded_preds_rouge, decoded_labels_rouge):
         if not pred or not ref:
              continue
         try:
            score = scorer.score(ref, pred)
            rouge_scores_collected['rouge1'].append(score['rouge1'].fmeasure)
            rouge_scores_collected['rouge2'].append(score['rouge2'].fmeasure)
            rouge_scores_collected['rougeL'].append(score['rougeL'].fmeasure)
         except Exception as e:
            print(f"Предупреждение: Ошибка при вычислении ROUGE для пары: '{ref}' vs '{pred}'. Ошибка: {e}")
            pass


    # Вычисляем среднее значение F1 для каждого типа ROUGE
    rouge1_avg = np.mean(rouge_scores_collected['rouge1']) if rouge_scores_collected['rouge1'] else 0.0
    rouge2_avg = np.mean(rouge_scores_collected['rouge2']) if rouge_scores_collected['rouge2'] else 0.0
    rougeL_avg = np.mean(rouge_scores_collected['rougeL']) if rouge_scores_collected['rougeL'] else 0.0

    result = {
        "f1": round(f1_score * 100, 4),
        "rouge1": round(rouge1_avg * 100, 4),
        "rouge2": round(rouge2_avg * 100, 4),
        "rougeL": round(rougeL_avg * 100, 4)
    }

    return result

In [49]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    report_to="none",
)

In [50]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipykernel_31/3505159916.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.002500
1000,0.001100
1500,0.000600
2000,0.001100
2500,0.001000
3000,0.000000
3500,0.000000
4000,0.000000


OSError: [Errno 28] No space left on device: './results/checkpoint-4080'

In [51]:
def evaluate_qa_model(test_dataset, model, tokenizer, num_samples=None, random_seed=42):
    """
    Функция для оценки метрик QA модели.
    """
    # Установка seed для воспроизводимости
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_seed)
    
    # Определение устройства
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")
    
    # Переводим модель на нужное устройство
    model = model.to(device)
    
    # Выбор примеров для оценки
    total_examples = len(test_dataset)
    if num_samples is None:
        num_samples = total_examples
    else:
        num_samples = min(num_samples, total_examples)
    
    indices = random.sample(range(total_examples), num_samples) if num_samples < total_examples else list(range(total_examples))
    
    # Инициализация ROUGE оценщика
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    # Подготовка хранилища для результатов
    results = {
        'indices': indices,
        'questions': [],
        'references': [],
        'predictions': [],
        'f1_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'confidence_scores': []
    }
    
    # Функция для вычисления F1-score на уровне токенов
    def calculate_token_f1(pred, ref):
        if not pred or not ref:
            return 0.0
        
        # Просто разделяем по пробелам для получения токенов
        pred_tokens = set(pred.lower().split())
        ref_tokens = set(ref.lower().split())
        
        if not pred_tokens or not ref_tokens:
            return 0.0
        
        common_tokens = pred_tokens.intersection(ref_tokens)
        
        precision = len(common_tokens) / len(pred_tokens) if len(pred_tokens) > 0 else 0.0
        recall = len(common_tokens) / len(ref_tokens) if len(ref_tokens) > 0 else 0.0
        
        if precision + recall == 0:
            return 0.0
        
        f1 = 2 * precision * recall / (precision + recall)
        return f1
    
    # Обработка примеров
    print(f"Оценка модели на {num_samples} примерах...")
    
    for idx in tqdm(indices, desc="Обработка примеров"):
        try:
            example = test_dataset[idx]
            question = example["question"]
            context = example["context"]
            
            # Проверка наличия эталонного ответа
            if "answers" in example and "text" in example["answers"] and len(example["answers"]["text"]) > 0:
                reference = example["answers"]["text"][0]
            else:
                print(f"Пример {idx} не содержит эталонного ответа. Пропускаем.")
                continue
            
            # Получаем ответ модели
            try:
                # Токенизация вопроса и контекста
                inputs = tokenizer(
                    question,
                    context,
                    return_tensors="pt",
                    max_length=384,
                    truncation="only_second",
                    padding="max_length"
                )
                
                # Переносим входные данные на устройство
                inputs = {k: v.to(device) for k, v in inputs.items()}
                
                # Получаем предсказания модели
                with torch.no_grad():
                    outputs = model(**inputs)
                
                # Находим наиболее вероятные начало и конец ответа
                answer_start = torch.argmax(outputs.start_logits)
                answer_end = torch.argmax(outputs.end_logits)
                
                # Корректируем индексы
                if answer_end < answer_start:
                    answer_end = answer_start
                
                # Декодируем ответ
                tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
                answer_tokens = tokens[answer_start:answer_end+1]
                predicted_answer = tokenizer.convert_tokens_to_string(answer_tokens)
                
                # Очищаем от специальных токенов
                for special_token in tokenizer.special_tokens_map.values():
                    if isinstance(special_token, str):
                        predicted_answer = predicted_answer.replace(special_token, "")
                    elif isinstance(special_token, list):
                        for token in special_token:
                            predicted_answer = predicted_answer.replace(token, "")
                
                predicted_answer = predicted_answer.strip()
                
                # Вычисляем уверенность
                start_probs = torch.softmax(outputs.start_logits, dim=1)
                end_probs = torch.softmax(outputs.end_logits, dim=1)
                confidence = start_probs[0, answer_start].item() * end_probs[0, answer_end].item()
                
                # Вычисляем F1-score
                f1 = calculate_token_f1(predicted_answer, reference)
                
                # Вычисляем ROUGE метрики
                rouge_scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
                if predicted_answer.strip() and reference.strip():
                    try:
                        score = scorer.score(reference, predicted_answer)
                        rouge_scores = {
                            "rouge1": score['rouge1'].fmeasure,
                            "rouge2": score['rouge2'].fmeasure,
                            "rougeL": score['rougeL'].fmeasure
                        }
                    except Exception as e:
                        print(f"Ошибка при вычислении ROUGE для примера {idx}: {e}")
                
                # Сохраняем результаты
                results['questions'].append(question)
                results['references'].append(reference)
                results['predictions'].append(predicted_answer)
                results['f1_scores'].append(f1)
                results['rouge1_scores'].append(rouge_scores["rouge1"])
                results['rouge2_scores'].append(rouge_scores["rouge2"])
                results['rougeL_scores'].append(rouge_scores["rougeL"])
                results['confidence_scores'].append(confidence)
                
            except Exception as e:
                print(f"Ошибка при обработке примера {idx}: {e}")
                
        except Exception as e:
            print(f"Ошибка при получении примера {idx}: {e}")
    
    # Вычисление агрегированных метрик
    if results['f1_scores']:
        avg_f1 = np.mean(results['f1_scores'])
        avg_rouge1 = np.mean(results['rouge1_scores'])
        avg_rouge2 = np.mean(results['rouge2_scores'])
        avg_rougeL = np.mean(results['rougeL_scores'])
        avg_confidence = np.mean(results['confidence_scores'])
        
        # Вывод результатов
        print("\nРезультаты оценки:")
        print(f"Всего обработано примеров: {len(results['f1_scores'])}")
        print(f"Средний F1-score: {avg_f1:.4f}")
        print(f"Средний ROUGE-1: {avg_rouge1:.4f}")
        print(f"Средний ROUGE-2: {avg_rouge2:.4f}")
        print(f"Средний ROUGE-L: {avg_rougeL:.4f}")
        print(f"Средняя уверенность модели: {avg_confidence:.4f}")
        
        # Показываем лучшие и худшие примеры
        sorted_indices = np.argsort(results['f1_scores'])
        
        print("\nПримеры с наихудшим F1-score:")
        for i in range(min(3, len(sorted_indices))):
            idx = sorted_indices[i]
            print(f"F1: {results['f1_scores'][idx]:.4f}")
            print(f"Вопрос: {results['questions'][idx]}")
            print(f"Эталон: {results['references'][idx]}")
            print(f"Предсказание: {results['predictions'][idx]}")
            print()
        
        print("\nПримеры с наилучшим F1-score:")
        for i in range(min(3, len(sorted_indices))):
            idx = sorted_indices[-(i+1)]
            print(f"F1: {results['f1_scores'][idx]:.4f}")
            print(f"Вопрос: {results['questions'][idx]}")
            print(f"Эталон: {results['references'][idx]}")
            print(f"Предсказание: {results['predictions'][idx]}")
            print()
        
        results['aggregated'] = {
            'avg_f1': avg_f1,
            'avg_rouge1': avg_rouge1,
            'avg_rouge2': avg_rouge2,
            'avg_rougeL': avg_rougeL,
            'avg_confidence': avg_confidence,
            'num_examples': len(results['f1_scores'])
        }
    
    return results

In [52]:
eval_results = evaluate_qa_model(
    test_dataset=test_dataset,
    model=model,
    tokenizer=tokenizer,
    num_samples=None
)

Используется устройство: cuda
Оценка модели на 1813 примерах...


Обработка примеров:   0%|          | 0/1813 [00:00<?, ?it/s]


Результаты оценки:
Всего обработано примеров: 1813
Средний F1-score: 0.6956
Средний ROUGE-1: 0.1772
Средний ROUGE-2: 0.0766
Средний ROUGE-L: 0.1772
Средняя уверенность модели: 0.9915

Примеры с наихудшим F1-score:
F1: 0.0000
Вопрос: Почему приятелю Ромашова всё показалось совершенно другим?
Эталон: Лёжа на спине, я незаметно уснул.
Предсказание: Почему приятелю Ромашова всё показалось совершенно другим

F1: 0.0000
Вопрос: Что именно заставило ежа ахнуть от удивления, вернувшись со сбора грибов?
Эталон: До самого вечера собирал ёжик грибы.
Предсказание: именно заставило ежа ахнуть от удивления, вернувшись

F1: 0.0000
Вопрос: Какой мужской предмет заметила Любовь Владимировна в руке у своего спутника, когда взяла его под руку?
Эталон: Так, наверно, они и дошли бы до тепла, если бы не этот проклятый ветер.
Предсказание: Какой мужской предмет заметила Любовь Владимировна в руке у своего спутника, когда взяла его под руку


Примеры с наилучшим F1-score:
F1: 1.0000
Вопрос: Стал ли обжаловат

In [53]:
model_path = "./rubert_qa_model"
trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('./rubert_qa_model/tokenizer_config.json',
 './rubert_qa_model/special_tokens_map.json',
 './rubert_qa_model/vocab.txt',
 './rubert_qa_model/added_tokens.json',
 './rubert_qa_model/tokenizer.json')